# 1. Cálculo Alternativo do ISDEL

In [84]:
# Importações e Configurações
import pandas as pd

pd.set_option('display.max_rows', 10)
pd.options.display.float_format = '{:.5f}'.format

In [85]:
# Leitura do CSV
df = pd.read_csv('subdim.csv', index_col=0, header=0)

## Anotação dos Pesos Default, modificar depois <br>

CAPITAL EMPREENDOR - 25% <br>
Densidade Empresarial - 20% <br>
Renda - 30% <br>
Educação - 50% <br>

TECIDO EMPRESARIAL - 10% <br>
Valores Solidários - programas e ações - 30% <br>
Tecido Social - 30% <br>
Tecido Empresarial - 40% <br>

GOVERNANÇA PARA O DESENVOLVIMENTO - 20% <br>
Articulação - 20% <br>
Gestão Pública - 40% <br>
Participação e Controle Social - 40% <br>

ORGANIZAÇÃO PRODUTIVA - 25% <br>
Vantagens Locais - 50% <br>
Organização Produtiva - 50% <br>

INSERÇÃO COMPETITIVA - 20% <br>
Comércio Internacional - 100% <br>

In [86]:
# Cálculo dos Pesos (Customizável)

# Modificar pesos (integers) conforme decisão do grupo
fce = 0.25
ce = [0.2, 0.3, 0.5]
ce = [element*fce for element in ce]

fte = 0.10
te = [0.3, 0.3, 0.4]
te = [element*fte for element in te]

fgd = 0.2
gd = [0.20, 0.40, 0.40]
gd = [element*fgd for element in gd]

fop = 0.25
op = [0.5, 0.5]
op = [element*fop for element in op]

fic = 0.2
ic = [1.0]
ic = [element*fic for element in ic]

# não modificar
vpesos = [ce, te, gd, op, ic]
pesos = [item for sublist in vpesos for item in sublist]
pesos

[0.05,
 0.075,
 0.125,
 0.03,
 0.03,
 0.04000000000000001,
 0.04000000000000001,
 0.08000000000000002,
 0.08000000000000002,
 0.125,
 0.125,
 0.2]

In [96]:
# Cálculo Alternativo do ISDEL

df_isdel = df.copy()

columns = list(df.columns)

n = len(columns)-1 # 12 colunas do index 0 a 11

#print('Contabilizando a coluna: ',columns[0])
df_isdel.loc[:,'ISDEL'] = pesos[0]*df_isdel.loc[:,columns[0]]

for i in range(0,n):
  #print('Contabilizando a coluna: ',columns[i+1])
  df_isdel.loc[:,'ISDEL'] = df_isdel.loc[:,'ISDEL'] + pesos[i+1]*df_isdel.loc[:,columns[i+1]]

df_isdel["ISDEL"]

isdel = df_isdel["ISDEL"]

isdel

Adamantina             0.42852
Adolfo                 0.34371
Aguaí                  0.33267
Águas da Prata         0.38898
Águas de Lindóia       0.38249
                         ...  
Vista Alegre do Alto   0.38273
Vitória Brasil         0.31480
Votorantim             0.43692
Votuporanga            0.45986
Zacarias               0.36930
Name: ISDEL, Length: 645, dtype: float64

# 2. Cálculo Alternativo do Promethee II

In [57]:
# Função Promethee II, copiada do Notebook
def compara(ai,aj,max=True):
  # Higher is better.
  if (max):
    res = 0
    if (ai > aj):
      res = 1
  # Lower is better.
  else:
    res = 1
    if (ai > aj):
      res = 0

  return res

# Cálculo de phi+ para 1 critério (coluna).
def tabelaC(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):
        res = compara(ai,aj,False)
      else:
        res = compara(ai,aj,True)
      aux.append(res)
    tc1.append(aux)

  return tc1


# Cálculo de phi- para 1 critério (coluna).
def tabelaCM(df,crit,min):
  tc1 = []
  for i in df.index:
    ai = df.loc[i,crit]
    aux = []
    # Comparação de ai com vários aj.
    for j in df.index:
      aj = df.loc[j,crit]
      if (min):
        res = compara(aj,ai,False)
      else:
        res = compara(aj,ai,True)
      aux.append(res)
    tc1.append(aux)

  return tc1

# Cálculo do Phi+ para todos os critérios (colunas).
def calcPhipPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns)

  #-----------------------------------------------------------
  # Cálculo do Phi+ (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc = tabelaC(df,columns[0],objetivos[0])
  dc = pesos[0]*pd.DataFrame(data=tc)
  print("Cálculo do critério "+columns[0])

  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci = tabelaC(df,columns[i],objetivos[i])
    dci = pesos[i]*pd.DataFrame(data=tci)
    print("Cálculo do critério "+columns[i])
    dc = dc.add(dci, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc.loc[:,'Total'] = dc.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc['Total'] = (dc['Total']/(m-1)).round(2)
  #-----------------------------------------------------------

  return dc['Total']


# Cálculo do Phi- para todos os critérios (colunas).
def calcPhinPesos(df,pesos,objetivos):

  # Obtendo o número de alternativas (linhas da tabela).
  rows = df.index
  m = len(rows)
  # Obtendo os critérios (colunas da tabela).
  columns = df.columns
  n = len(columns)

  #-----------------------------------------------------------
  # Cálculo do Phi- (todas as colunas).
  #-----------------------------------------------------------
  # Comparando as alternativas em relação ao primeiro critério.
  tc2 = tabelaCM(df,columns[0],objetivos[0])
  dc2 = pesos[0]*pd.DataFrame(data=tc2)
  print("Cálculo do critério "+columns[0])

  # Comparando as alternativas nos demais critérios e armazenado
  # o resultado acumulado em dc.
  for i in range(1,n):
    tci2 = tabelaCM(df,columns[i],objetivos[i])
    dci2 = pesos[i]*pd.DataFrame(data=tci2)
    print("Cálculo do critério "+columns[i])
    dc2 = dc2.add(dci2, fill_value=0)

  # Resultando final: para cada linha soma das colunas.
  dc2.loc[:,'Total'] = dc2.sum(axis = 1, skipna = True)
  # Dividir por 1/(m-1) a soma total.
  dc2['Total'] = (dc2['Total']/(m-1)).round(2)

  return dc2['Total']



# Cálculo do índice de ranqueamento do Promethee II.
def prometheeIIpesos(df,pesos,objetivos):
  print("Iniciando cálculos do Promethee II...")

  # Cálculo do Phi+.
  print('Cálculos do Phi+...')
  dcp = calcPhipPesos(df,pesos,objetivos)

  # Cálculo do Phi-.
  print('Cálculos do Phi-...')
  dcn = calcPhinPesos(df,pesos,objetivos)

  print('Cálculos do Phi = Phi+ - Phi-...')

  return dcp-dcn

In [74]:
# Cálculo Alternativo do Promethee II

#já temos os pesos ponderados, obtidos no cálculo do ISDEL, temos que setar os objetivos
objetivos = [False]*len(pesos)

#trazer a df_isdel e retirar coluna do ISDEL
df_phi = df_isdel.loc[:,(df_isdel.columns != "ISDEL")]

rankvalue = prometheeIIpesos(df_phi, pesos, objetivos)

rankvalue

Iniciando cálculos do Promethee II...
Cálculos do Phi+...
Cálculo do critério Densidade Empresarial
Cálculo do critério Renda
Cálculo do critério Educação
Cálculo do critério Valores Solidários - programas e ações
Cálculo do critério Tecido Social
Cálculo do critério Tecido Empresarial
Cálculo do critério Articulação
Cálculo do critério Gestão Pública
Cálculo do critério Participação e Controle Social
Cálculo do critério Vantagens Locais
Cálculo do critério Organização Produtiva
Cálculo do critério Comércio Internacional
Cálculos do Phi-...
Cálculo do critério Densidade Empresarial
Cálculo do critério Renda
Cálculo do critério Educação
Cálculo do critério Valores Solidários - programas e ações
Cálculo do critério Tecido Social
Cálculo do critério Tecido Empresarial
Cálculo do critério Articulação
Cálculo do critério Gestão Pública
Cálculo do critério Participação e Controle Social
Cálculo do critério Vantagens Locais
Cálculo do critério Organização Produtiva
Cálculo do critério Comérci

0      0.37000
1     -0.14000
2     -0.20000
3      0.16000
4      0.07000
        ...   
640    0.25000
641   -0.42000
642    0.57000
643    0.69000
644   -0.04000
Name: Total, Length: 645, dtype: float64

In [95]:
# Consolidação dos Resultados em um DF

# Consolidando tudo em um DF novo
df_consolidada = df.copy()
df_consolidada.loc[:,'ISDEL'] = isdel
df_consolidada.loc[:,'PrometheeII'] = rankvalue.values # corrigindo o problema do index

#Ascending Sort
df_consolidada = df_consolidada.sort_values('PrometheeII',ascending=False)
df_consolidada.head(20)

,Densidade Empresarial,Renda,Educação,Valores Solidários - programas e ações,Tecido Social,Tecido Empresarial,Articulação,Gestão Pública,Participação e Controle Social,Vantagens Locais,Organização Produtiva,Comércio Internacional,ISDEL,PrometheeII
Santos,0.32920,0.89491,0.89517,1.00000,0.09975,0.01841,0.08333,0.82264,0.67063,0.48508,0.61427,0.27439,0.54430,0.91000
Barueri,0.35862,0.72069,0.85651,0.00000,0.02945,0.04584,1.00000,0.87986,0.62881,0.45527,0.81588,0.30906,0.56316,0.90000
Campinas,0.31616,0.81227,0.85155,0.85714,0.06705,0.05544,0.08333,0.80157,0.74833,0.49095,0.87274,0.34055,0.57901,0.88000
São Caetano do Sul,0.40281,0.99669,0.94246,1.00000,0.00631,0.00220,1.00000,0.77479,0.50079,0.54999,0.59871,0.14893,0.55840,0.88000
Sorocaba,0.29258,0.73900,0.86748,1.00000,0.03270,0.01181,0.16667,0.85670,0.76984,0.42846,0.67679,0.22123,0.52913,0.88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
São José do Rio Preto,0.35394,0.76445,0.86906,0.42857,0.03326,0.00661,0.00000,0.70897,0.65944,0.45166,0.59628,0.18251,0.47475,0.81000
Indaiatuba,0.25597,0.74817,0.86891,0.00000,0.00943,0.00080,0.25000,0.92769,0.68651,0.42862,0.62088,0.15451,0.47907,0.80000
Rio Claro,0.25583,0.72766,0.87107,0.85714,0.01019,0.00080,0.25000,0.70539,0.61563,0.43594,0.57373,0.22576,0.48934,0.80000
Santana de Parnaíba,0.44024,0.92252,0.87368,0.42857,0.02082,0.00520,0.58333,0.81426,0.55135,0.37869,0.60826,0.15345,0.50074,0.79000


# 3. Geolocalização Estado de São Paulo - IBGE

API Localidades <br>
https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios <br>

API Malhas Geográficas <br>
https://servicodados.ibge.gov.br/api/v3/malhas/estados/35?formato=image/svg+xml&qualidade=maxima&intrarregiao=municipio <br>
https://servicodados.ibge.gov.br/api/v3/malhas/estados/35?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=municipio <br>